In [ ]:
# 模型导出，c++/python高性能推理（可以认为只具有前向推理）

In [ ]:
# 1.onnx及其onnxruntime（微软）
# 只要平台或终端支持onnxruntime就可以执行onnx模型
# 无论通过何种方式导出ONNX模型，最终的目的都是将模型部署到目标平台并进行推理。
# 目前为止，很多推理框架都直接或者间接的支持ONNX模型推理，如ONNXRuntime（ORT）、TensorRT和TVM
# （TensorRT和TVM将在后面的文章中进行介绍与分析）可以直接部署ONNX模型，Torch、Tensorflow和mxnet等可以间接的
# 通过官方提供的工具对模型进行转换实现ONN模型的部署。
# onnx底层采用谷歌的protobuffer定义
# onnx与pytorch交互本质是调用torch.jit.trace

pip install onnx
pip install onnxruntime # use cpu
pip install onnxruntime-gpu # use gpu


In [40]:
import torch
import torch.onnx
from time import time
device = torch.device("cuda:0" if torch.cuda.is_available() else ('mps:0' if torch.backends.mps.is_available() else "cpu"))

model = torch.load('models/LSTM_MLP_AE_[64,32],[32,64]_32_epoch=997_valloss=234.79.pt')

# 必须为推理模式
model.eval()
# dummy_input就是一个输入的实例，仅提供输入shape、type等信息 
batch_size = 10 # 随机的取值，当设置dynamic_axes后影响不大
dummy_input = torch.randn(batch_size, 30, 3, requires_grad=True).to(device)
t1 = time()
output = model(dummy_input)
print('time:', time()-t1)

# 导出模型，实现从pytorch模型到onnx转换，每个pytorch操作转换onnx算子
onnx_file_name = 'models/LSTM_MLP_AE.onnx'
torch.onnx.export(model,        # 模型的名称
                  dummy_input,   # 一组实例化输入
                  onnx_file_name,   # 文件保存路径/名称
                  export_params=True,        #  如果指定为True或默认, 参数也会被导出. 如果你要导出一个没训练过的就设为 False.
                  opset_version=10,          # ONNX 算子集的版本，当前已更新到15
                  do_constant_folding=True,  # 是否执行常量折叠优化
                  input_names = ['input'],   # 输入模型的张量的名称，需要用netron查看onnx文件
                  output_names = ['output'], # 输出模型的张量的名称，需要用netron查看onnx文件
                  # dynamic_axes将batch_size的维度指定为动态，
                  # 后续进行推理的数据可以与导出的dummy_input的batch_size不同
                  dynamic_axes={'input' : {0 : 'batch_size'},
                                'output' : {0 : 'batch_size'}})

# 实际上，导出需要注意很多问题，模型里一些算子转化为onnx并不友好，需要修改

time: 0.00858306884765625
================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [33]:
import onnx
# 我们可以使用异常处理的方法进行检验
try:
    # 当我们的模型不可用时，将会报出异常
    onnx.checker.check_model(onnx_file_name)
except onnx.checker.ValidationError as e:
    print("The model is invalid: %s"%e)
else:
    # 模型可用时，将不会报出异常，并会输出“The model is valid!”
    print("The model is valid!")


The model is valid!


In [ ]:
# 采用Netron可视化ONNX格式的模型
# https://github.com/lutzroeder/netron/releases/tag/v7.1.3


In [41]:
# 采用onnx runtime推理引擎对onnx模型进行推理（微软研制）
import onnxruntime
# 需要进行推理的onnx模型文件名称

# onnxruntime.InferenceSession用于获取一个 ONNX Runtime 推理器
ort_session = onnxruntime.InferenceSession(onnx_file_name)  

# 构建字典的输入数据，字典的key需要与我们构建onnx模型时的input_names相同
# 输入也需要改变为ndarray格式
ort_inputs = {'input': dummy_input.detach().cpu().numpy()}
# 我们更建议使用下面这种方法,因为避免了手动输入key
# ort_inputs = {ort_session.get_inputs()[0].name:input_img}

# pytorch动态建模，故需要执行一遍
# run是进行模型的推理，第一个参数为输出张量名的列表，一般情况可以设置为None
# 第二个参数为构建的输入值的字典
# 由于返回的结果被列表嵌套，因此我们需要进行[0]的索引
t1 = time()
ort_output = ort_session.run(None,ort_inputs)[0]
print('time:', time()-t1)
# output = {ort_session.get_outputs()[0].name}
# ort_output = ort_session.run([output], ort_inputs)[0]

print('input:', dummy_input.detach().cpu().numpy().shape)
print('output:', ort_output.shape)
print(ort_output)

# onnx比 pytorch model推理速度快数倍，这里是10倍
# ONNX 表示更容易部署的静态图

time: 0.0011768341064453125
input: (10, 30, 3)
output: (10, 30, 3)
[[[3.23216617e-02 3.31142545e-03 1.52625173e-01]
  [1.56924307e-01 2.24587321e-03 1.58904791e-01]
  [1.63386464e-01 3.64223123e-03 1.45755202e-01]
  [2.97241092e-01 7.05817640e-02 1.43262357e-01]
  [3.34667712e-01 7.13166296e-02 1.37585998e-01]
  [3.64846259e-01 1.54044390e-01 1.37276560e-01]
  [4.27440286e-01 2.05454558e-01 1.51976436e-01]
  [4.19373035e-01 2.71701992e-01 1.46220148e-01]
  [5.05093336e-01 2.58284658e-01 1.64969146e-01]
  [4.57483113e-01 2.53103524e-01 1.48771316e-01]
  [5.32694817e-01 2.10138977e-01 1.44022912e-01]
  [5.63353300e-01 2.24701196e-01 1.55759186e-01]
  [5.74124515e-01 8.48690569e-02 1.57262921e-01]
  [5.78734219e-01 2.47707069e-02 1.54802412e-01]
  [5.81529021e-01 1.28314793e-02 1.61350995e-01]
  [6.16788208e-01 1.08636320e-02 1.69775695e-01]
  [5.82189441e-01 7.64288604e-02 1.67744070e-01]
  [5.11988461e-01 2.44493246e-01 1.44173056e-01]
  [4.59290028e-01 4.27703440e-01 1.46017939e-01]
  

In [42]:
# onnx模型量化(三行代码)
# 模型大小减小近乎一半
# pip install onnxconverter_common

from onnxconverter_common import float16

model_ = onnx.load('models/LSTM_MLP_AE.onnx') # 409 kb
model__ = float16.convert_float_to_float16(model_)
onnx.save(model__, 'models/LSTM_MLP_AE_.onnx') # 210 kb


In [54]:
# 2.torchscript（pytorch自己的-python版本部署）:  script(pytorch model) or trace(pytorch model, input)，pytorch自己的静态存储，仅用于推理
# 采用无输入的torch.jit.script,源码转换
# 支持if-else等控制流，支持非tensor的数据类型，需要对部分数据进行类型标注（：）
model_1 = torch.jit.script(model)
print(1,model_1.code)
print(2,model_1.graph)
# 完全确定子模块输出，所有子模块内联
torch._C._jit_pass_inline(model_1.graph)
print(3,model_1.code)
t1 = time()
output = model_1(dummy_input)
print('time:', time()-t1)
torch.jit.save(model_1, 'models/LSTM_MLP_AE_jittest.pt')
# torch.jit.load('models/LSTM_MLP_AE_jittest.pt') # 暂时不支持mps，RuntimeError: supported devices include CPU, CUDA and HPU, however got MPS

# 采用有输入的torch.jit.trace
# 不支持if-else控制流，不支持tensor以外的数据类型
model_2 = torch.jit.trace(model, dummy_input)
print(model_2.code)
t1 = time()
output = model_1(dummy_input)
print('time:', time()-t1)


1 def forward(self,
    x: Tensor) -> Tensor:
  _0 = "LSTM: Expected input to be 2-D or 3-D but received {}-D tensor"
  _1 = "input must have {} dimensions, got {}"
  _2 = "input.size(-1) must be equal to input_size. Expected {}, got {}"
  _3 = "LSTM: Expected input to be 2-D or 3-D but received {}-D tensor"
  _4 = "input must have {} dimensions, got {}"
  _5 = "input.size(-1) must be equal to input_size. Expected {}, got {}"
  decoder = self.decoder
  encoder = self.encoder
  input_layer = encoder.input_layer
  seq_len = encoder.seq_len
  input_dim = encoder.input_dim
  _6 = torch.reshape(x, [-1, seq_len, input_dim])
  _00 = getattr(input_layer, "0")
  weight = _00.weight
  bias = _00.bias
  input = torch.linear(_6, weight, bias)
  x0 = torch.tanh(input)
  lstms = encoder.lstms
  _01 = getattr(lstms, "0")
  _7 = torch.dim(x0)
  if torch.__contains__([2, 3], _7):
    pass
  else:
    _8 = torch.add("AssertionError: ", torch.format(_0, torch.dim(x0)))
    ops.prim.RaiseException(_8)
  i

/Users/fubin/opt/anaconda3/lib/python3.9/site-packages/torch/jit/_trace.py:154: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/build/aten/src/ATen/core/TensorBody.h:491.)
  if a.grad is not None:


In [ ]:
# 时间对比
# pytorch model: 0.00858306884765625
# onnxruntime: 0.0011768341064453125
# torch.jit.script: 0.0030870437622070312
# torch.jit.trace: 0.0023758411407470703


In [ ]:
# 3.libtorch（pytorch自己的-c++版本）: c++里的pytorch，实现c++环境下的部署
# 这里省略，需要在c++中安装相应包，并执行c++环境，如pytorch中的操作torch.einsum对应libtorch中的torch::einsum
# 1.可以直接用libtorch来实现模型
# 2.可以直接加载ONNX或者pt模型文件

# 以下为c++代码，其实风格同pytorch类似:
#include <torch/torch.h>
#include <iostream>
using namespace std;

int main(int argc, const char ** argv){
    torch::Tensor x = torch::randn((2,3));
    torch::nn::Linear f(3,6);
    # torch::Tensor y = f(x);
    torch::Tensor y = f->forward(x);
    cout << y << endl;
}

In [ ]:
# 4.TensorRT 3（NVIDIA自家GPU优化）：基于ONNX提供c++/python api，工业级推理方案
# nvidia发布dnn推理引擎，针对nvidia硬件进行优化加速，实现最大化利用gpu资源，提升推理性能
# 是业内nvidia系列产品部署落地时的最佳选择，基于libnvonxparser.so（ONNXParser）
# 高性能后续需要考虑，高吞吐、低延迟、低内存设备占用，异步进行、消除没用的操作
# onnx模型 -> TRT模型
# 执行过程：onnx文件  ->  trt api  ->  trt builder  ->  trt engine
# 比cpu快40倍

# c++：
TRT::compile(
    TRT::Mode::FP32, # 使用FP32模型编译
    3, # max batch size
    "plugin.onnx", // onnx文件
    "plugin.fp32.trtmodel", // 保存文件路径
    {}
)

# python：
# pip install trtpy
import trtpy
model = models.resnet18(True).eval()
trtpy.from_torch(
    model,
    max_batch_size=16,
    onnx_save_file='LSTM_MLP_AE.onnx',
    engine_save_file='LSTM_MLP_AE.trtmodel'
)

# 

In [ ]:
# 5.基于flask web部署

